#### Imports

In [ ]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup

### > Tarefa 1

#### 1.1 - Crawler

In [ ]:
# Função que salva a nossa página html na pasta "paginas" junto ao número da página
def salva_pagina(num_pag, html):
    with open(f"tarefa1/paginas/{num_pag}.html", "w", encoding="utf-8") as arq:
        arq.write(html)

# Função que busca a próxima página pelo botão "Next >"
def encontra_proxima_pag(req):
    pag_bs = BeautifulSoup(req.text)
    url = pag_bs.find('a', string=["Next >"]).get('href')
    
    return "http://127.0.0.1:8000" + url

# Define o nosso crawler, que navega entre as páginas recursivamente salvando o .html delas
def crawler(url, num_pag=0):
    #print(url)
    req = requests.get(url)

    soup = BeautifulSoup(req.text)
    link_paises = [i.get('href') for i in soup.select("div[id='results'] > table > tbody > tr > td > div > a")]

    for pais in link_paises:
        req_pais = requests.get("http://127.0.0.1:8000" + pais)
        salva_pagina(num_pag, req_pais.text)
        num_pag+=1

    try:
        return crawler(encontra_proxima_pag(req), num_pag)
    except:
        return None

In [ ]:
# Executando o Crawler

url = "http://127.0.0.1:8000/places/default/index/0"
crawler(url)

#### 1.2 - "Page" Scraper

In [ ]:
# Função que retorna uma página salva em meio aos arquivos que o Crawler coletou
def abre_pagina(num_pag):
    with open(f"tarefa1/paginas/{num_pag}.html", "r", encoding="utf-8") as arq:
        return arq.readlines()

# Função que abstrai o padrão de tag da maior parte dos atributos do nosso scraping
def cata_atributo(bs, id):
    return bs.find('tr', id=id).find('td', class_='w2p_fw')

# Faz a "raspagem" dos dados, abrindo pelos hrefs referentes aos países daquela página e empilhando os atributos coletados de cada país
def scrap_pagina(pag):
    pais_bs = BeautifulSoup(str(pag))

    # String que vai guardar os atributos dos países disponíveis nessa página
    scrap_paises = ""

    # Sub-processo para pegar o nome completo do continente, acessando a página do continente pela sua sigla
    conti_url = "http://127.0.0.1:8000" + cata_atributo(pais_bs, 'places_continent__row').find('a').get('href')
    conti_req = requests.get(conti_url)
    
    conti_bs = BeautifulSoup(conti_req.text)

    # Atributos
    dt_hr_update = str(datetime.now())[:19].replace(' ', 'T') # Formata o nosso timestamp de forma "mais light"
    nm_pais = cata_atributo(pais_bs, 'places_country__row').text
    nm_capital = cata_atributo(pais_bs, 'places_capital__row').text
    nm_currency = cata_atributo(pais_bs, 'places_currency_name__row').text
    nm_continente = conti_bs.find('section', id='main').find('h2').text

    # Agrupa os atributos na string de scrap, em formato compatível com o .csv
    scrap_paises += dt_hr_update+";"+nm_pais+";"+nm_capital+";"+nm_currency+";"+nm_continente+"\n"

    try:
        return scrap_paises + crawler(encontra_proxima_pag(req), num_pag)
    except:
        return scrap_paises

In [ ]:
# Gerando o dataset
dataset_paises = 'dt_hr_update;nm_pais;nm_capital;nm_currency;nm_continente\n'
for pag in range(252):
    dataset_paises += scrap_pagina(abre_pagina(pag))

# Remove a última linha (vazia)
dataset_paises = dataset_paises[:-1]

# Salva a variavel str como dataset em formato csv
with open("tarefa1/dataset_paises.csv", "w", encoding="utf-8") as arq:
    arq.write(dataset_paises)

#### 1.3 - Monitor de arquivos

In [ ]:
# Executando o Crawler novamente em busca de atualizacoes

url = "http://127.0.0.1:8000/places/default/index/0"
crawler(url)

# Abre o dataset previamente salvo
dataset_paises = []
with open("tarefa1/dataset_paises.csv", "r", encoding="utf-8") as arq:
    dataset_paises = arq.readlines()

# Copia o dataset para comparacoes futuras
dataset_original = dataset_paises.copy()

# Itera em cima das paginas que estao sendo checadas, comparando/modificando com a versao presente no dataset
for pag in range(252):
    pagina_staged = scrap_pagina(abre_pagina(pag))
    pagina_dataset = dataset_paises[pag+1]

    # Compara o arquivo mais recente com os dados do dataset (sem comparar a data/hora de update ou '\n')
    if pagina_dataset.split('\n')[0].split(';')[1:] != pagina_staged.split('\n')[0].split(';')[1:]:
        dataset_paises[pag+1] = pagina_staged

# Salva o dataset caso atualizacoes tenham acontecido
if dataset_paises != dataset_original:
    dataset_atualizado = ''

    # Convertendo a lista em string
    for i in dataset_paises:
        dataset_atualizado += i

    with open("tarefa1/dataset_paises.csv", "w", encoding="utf-8") as arq:
        arq.write(dataset_atualizado)